# **Computational Drug Discovery dardarin Download Bioactivity Data**

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [2]:
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 7.0 MB/s eta 0:00:00
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.1.2 which is incompatible.


## **Importing libraries**

In [3]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for** **dardarin**

In [4]:
# Target search for dardarin 
target = new_client.target
target_query = target.search('dardarin ')


In [5]:
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'Q5S007', 'xref_name': None, 'xre...",Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,18.0,False,CHEMBL1075104,"[{'accession': 'Q5S007', 'component_descriptio...",SINGLE PROTEIN,9606


### **Select and retrieve bioactivity data for dardarin**

We will assign the 1st entry (which corresponds to the **selected_target** variable 

In [6]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL1075104'

Here, we will retrieve only bioactivity data for *Leucine-rich repeat serine/threonine-protein kinase* (CHEMBL1075104) that are reported as IC$_{50}$ values in nM (nanomolar) unit.

In [7]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [8]:
df = pd.DataFrame.from_dict(res)

In [9]:
df.head(3)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,6180232,[],CHEMBL1772597,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,13.2
1,None,6180234,[],CHEMBL1772599,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,4.1
2,None,7989381,[],CHEMBL1937075,Inhibition of human LRRK2,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,16.0


In [10]:
df.standard_type.unique()

array(['IC50'], dtype=object)

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [11]:
df.to_csv('dardarin_biological_activity_preprocessed_data.csv', index=False)

## **Copying files to Google Drive**

Firstly, we need to mount the Google Drive into Colab so that we can have access to our Google adrive from within Colab.

In [12]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


Mounted at /content/gdrive/


Next, we create a **data** folder in our **Colab Notebooks** folder on Google Drive.

In [13]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data2"

mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab Notebooks/data2’: File exists


In [14]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

cp: cannot stat 'bioactivity_data.csv': No such file or directory


In [15]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

-rw------- 1 root root 84793 Feb 12 18:05 '/content/gdrive/My Drive/Colab Notebooks/data'


Let's see the CSV files that we have so far.

In [16]:
! ls

dardarin_biological_activity_preprocessed_data.csv  drive  gdrive  sample_data


Taking a glimpse of the **bioactivity_data.csv** file that we've just created.

In [17]:
! head dardarin_preprocessed_data.csv

head: cannot open 'dardarin_preprocessed_data.csv' for reading: No such file or directory


## **Handling missing data**
If any compounds has missing value for the **standard_value** column then drop it

In [18]:
df2 = df[df.standard_value.notna()]
df2

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,6180232,[],CHEMBL1772597,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,13.2
1,None,6180234,[],CHEMBL1772599,Inhibition of purified N-terminal GST-tagged L...,B,Q5S007,G2019S,BAO_0000190,BAO_0000019,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,uM,UO_0000065,None,4.1
2,None,7989381,[],CHEMBL1937075,Inhibition of human LRRK2,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,16.0
3,None,10840480,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,13.0
4,None,10840481,[],CHEMBL2015541,Inhibition of wild-type LRRK2 expressed in HEK...,B,None,None,BAO_0000190,BAO_0000219,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2072,Active,23352403,[],CHEMBL4882639,Cellular assay monitoring dephosphorylation of...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,3.5
2073,Not Active,23352405,[],CHEMBL4882637,LRRK2 Biochemical Assay,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,6480.0
2074,Not Active,23352406,[],CHEMBL4882641,Cellular assay monitoring dephosphorylation of...,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,IC50,nM,UO_0000065,None,5439.0
2075,None,23373725,"[{'comments': None, 'relation': None, 'result_...",CHEMBL4885502,LRRK2(LRR2PGF1) Takeda global kinase panel,B,None,None,BAO_0000190,BAO_0000357,...,Homo sapiens,Leucine-rich repeat serine/threonine-protein k...,9606,None,None,pIC50,None,UO_0000065,None,6.0


Apparently, for this dataset there is no missing data. But we can use the above code cell for bioactivity data of other target protein.

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [19]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Iterate the *molecule_chembl_id* to a list**

In [20]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

### **Iterate *canonical_smiles* to a list**

In [21]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

### **Iterate *standard_value* to a list**

In [22]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

### **Combine the 4 lists into a dataframe**

In [23]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [24]:
df3

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value
0,CHEMBL1771409,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,inactive,13200.0
1,CHEMBL1771411,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,intermediate,4100.0
2,CHEMBL1933288,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,active,16.0
3,CHEMBL2012582,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,active,13.0
4,CHEMBL509032,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,active,7.8
...,...,...,...,...
2015,CHEMBL4098877,C[C@H]1CN(c2cc(-c3n[nH]c4ccc(OC5(C)CC5)cc34)nc...,active,3.5
2016,CHEMBL4560589,C[C@H]1CN(c2cc(-c3nn(C)c4ccc(OC5(C)CC5)cc34)nc...,intermediate,6480.0
2017,CHEMBL4560589,C[C@H]1CN(c2cc(-c3nn(C)c4ccc(OC5(C)CC5)cc34)nc...,intermediate,5439.0
2018,CHEMBL4097778,CN1C(=O)[C@@H](N2CCc3cn(CC4CCS(=O)(=O)CC4)nc3C...,active,1000.0


In [25]:
pd.concat([df3,pd.Series(bioactivity_class)], axis=1)

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value,0
0,CHEMBL1771409,Cc1cc(N/N=C/c2ccc(O)c(O)c2)nc2ccccc12,inactive,13200.0,inactive
1,CHEMBL1771411,Cc1cc(N/N=C/c2ccncc2)nc2ccccc12,intermediate,4100.0,intermediate
2,CHEMBL1933288,C[C@@H]1CCNC(=O)c2cc3ccc(C(=O)Nc4nc5ccccc5n4CC...,active,16.0,active
3,CHEMBL2012582,COc1cc(C(=O)N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc2c...,active,13.0,active
4,CHEMBL509032,COc1cc(N2CCC(N3CCN(C)CC3)CC2)ccc1Nc1ncc(Cl)c(N...,active,7.8,active
...,...,...,...,...,...
2015,CHEMBL4098877,C[C@H]1CN(c2cc(-c3n[nH]c4ccc(OC5(C)CC5)cc34)nc...,active,3.5,active
2016,CHEMBL4560589,C[C@H]1CN(c2cc(-c3nn(C)c4ccc(OC5(C)CC5)cc34)nc...,intermediate,6480.0,intermediate
2017,CHEMBL4560589,C[C@H]1CN(c2cc(-c3nn(C)c4ccc(OC5(C)CC5)cc34)nc...,intermediate,5439.0,intermediate
2018,CHEMBL4097778,CN1C(=O)[C@@H](N2CCc3cn(CC4CCS(=O)(=O)CC4)nc3C...,active,1000.0,active


Saves dataframe to CSV file

In [26]:
df3.to_csv('dardarin_bioactivity_preprocessed_data.csv', index=False)

In [27]:
! ls -l

ls: cannot access 'drive': Transport endpoint is not connected
total 2244
-rw-r--r-- 1 root root  151211 Feb 18 10:19 dardarin_bioactivity_preprocessed_data.csv
-rw-r--r-- 1 root root 2137974 Feb 18 10:19 dardarin_biological_activity_preprocessed_data.csv
d????????? ? ?    ?          ?            ? drive
drwx------ 5 root root    4096 Feb 18 10:19 gdrive
drwxr-xr-x 1 root root    4096 Feb 17 19:30 sample_data


Let's copy to the Google Drive

In [28]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

cp: cannot stat 'bioactivity_preprocessed_data.csv': No such file or directory


In [29]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"

'/content/gdrive/My Drive/Colab Notebooks/data'


---